In [3]:
%pip install datasets

  Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 3.7 MB/s eta 

In [4]:
import datasets
import json
import os
import math

/Users/vh19/.local/share/mise/installs/python/3.13.5/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Change these variables to perform evalution for you task
original_quiz_path = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + '/processed_data/gpt3/completion_4/processed_test.json'
generated_quiz_path = os.path.abspath(os.getcwd()).split('gpt5_evaluation_scripts')[0] + '/generated_data_gpt5/EEQG/generated_quiz.json'

In [13]:
# Get original quizzes
original_quizzes = []

for line in open(original_quiz_path):
    original_quizzes.append(json.loads(line)['completion'].split("\n###")[0].replace("\n"," ").strip())
    
# Get generated quizzes
generated_quizzes = []
with open(generated_quiz_path) as f:
    generated_quizzes_dict = json.load(f)

for key in generated_quizzes_dict:
    generated_quizzes.append(generated_quizzes_dict[key].replace("\n"," ").strip())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/vh19/Documents/Works/personal-projects/EduQuiz-vh//generated_data_gpt5/EEQG/generated_quiz.json'

In [4]:
# Put the predictions and gold references in lists

predictions = []
gold_references = []
predictions_list = []
gold_references_list = []

for i in range(len(original_quizzes)):
    predictions.append(generated_quizzes[i])
    gold_references.append(original_quizzes[i])

    predictions_list.append(generated_quizzes[i].split(' '))
    gold_references_list.append([original_quizzes[i].split(' ')])

In [5]:
bleu = datasets.load_metric('bleu')
rouge = datasets.load_metric('rouge')

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
bleu.add_batch(predictions=predictions_list, references=gold_references_list)
rouge.add_batch(predictions=predictions, references=gold_references)

In [7]:
final_bleu = bleu.compute()
final_rouge = rouge.compute()

In [8]:
# Define path of downloaded meteor from https://www.cs.cmu.edu/~alavie/METEOR/
meteor_path = "<PATH_TO_YOUR_DOWNLOADED_METEOR>"

# Move results to meteor directory
with open(meteor_path + "predictions.txt", 'w') as f:
    for i in range(len(predictions)):
        f.write(str(predictions[i]) + '\n')
        
with open(meteor_path + "ground_truth.txt", 'w') as f:
    for i in range(len(gold_references)):
        f.write(str(gold_references[i]) + '\n')

# Run the meteor command from the meteor directory and remove result files again   
wd = os.getcwd()
os.chdir(meteor_path)
output = os.popen("java -Xmx2G -jar meteor-*.jar predictions.txt ground_truth.txt -l en -norm").read()
os.remove(meteor_path + "predictions.txt")
os.remove(meteor_path + "ground_truth.txt")
os.chdir(wd)

# Get the score from the output
meteor_score = round(float(output.split("Final score:")[1].strip()) * 100, 2)

In [9]:
print("BLEU: ", str(round(final_bleu['bleu'] * 100, 2)))
print("ROUGE-L: ", str(round(final_rouge['rougeL'].mid.fmeasure * 100, 2)))
print("METEOR: ", str(meteor_score))

BLEU:  11.61
ROUGE-L:  36.11
METEOR:  25.42


In [10]:
# Create result files
        
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'generated_data_gpt3/EEQG/' + 'automatic_evaluation_EEQG.txt', 'w') as f:
    f.write("BLEU: " + str(round(final_bleu['bleu'] * 100, 2)))
    f.write('\n')
    f.write("ROUGE-L: " + str(round(final_rouge['rougeL'].mid.fmeasure * 100, 2)))
    f.write('\n')
    f.write("METEOR: " + str(meteor_score))

In [11]:
# Get all the target prompts and completions
test_data = []

for line in open(original_quiz_path):
    test_data.append((json.loads(line)))

# Get quizzes
quizzes = []
with open(generated_quiz_path) as f:
    gen_quizzes = json.load(f)

for key in gen_quizzes:
    quizzes.append(gen_quizzes[key])

count = 0
with open(os.path.abspath(os.getcwd()).split('gpt3_evaluation_scripts')[0] + 'human_evaluation/EEQG/' + 'EEQG_gpt3.txt', 'w') as f:
    for i in range(0,len(test_data),math.floor(len(test_data)/100)):
        f.write('Excel row: ' + str(count+2) + ' Test instance: ' + str(i+1) + '\n\n')
        f.write(test_data[i]['prompt'].split("\n\n###")[0] + '\n\n')
        f.write('Generated quiz:\n')
        f.write(quizzes[i].strip() + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break